In [ ]:
# IMPORTANT NOTES BEFORE USE
# 1. This notebook should be run under a linux environment with a NVIDIA GPU. WSL2 is recommended.
# 2. CUDA 12.0 or higher is required. See https://docs.rapids.ai/install/#system-recommendations for more details.
# 3. Latest CUDA toolkit is recommended. See https://developer.nvidia.com/cuda-downloads for more details.
# 4. Python 3.10, 3.11, or 3.12 is required.
# 5. As referred to the RAPIDS documentation, RAPIDS pip packages require a recent version of pip that supports PEP600. If you encountered any problem, try upgrade pip with: pip install -U pip.
#       See https://docs.rapids.ai/install/#troubleshooting for more details.
# 6. It is STRONGLY RECOMMENDED to use a virtual environment (e.g., venv) to avoid package conflicts.

# Install dependencies
%pip install \
    --extra-index-url=https://pypi.nvidia.com \
    "cudf-cu12==25.4.*" "cuml-cu12==25.4.*"
%pip install opencv-python
%pip install numpy
%pip install pandas
%pip install scikit-image
%pip install scikit-learn

Looking in indexes: https://pypi.org/simple, https://pypi.nvidia.com
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 1.9 MB/s eta 0:00:00a 0:00:010m
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.7/9.7 MB 9.4 MB/s eta 0:00:00a 0:00:01
  Using cached cachetools-5.5.2-py3-none-any.whl.metadata (5.4 kB)
  Using cached cuda_python-12.8.0-py3-none-any.whl.metadata (15 kB)
  Using cached cupy_cuda12x-13.4.1-cp312-cp312-manylinux2014_x86_64.whl.metadata (2.6 kB)
  Using cached fsspec-2025.3.2-py3-none-any.whl.metadata (11 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 565.0/565.0 MB 43.8 MB/s eta 0:00:0000:0100:01
  Using cached numba_cuda-0.4.0-py3-none-any.whl.metadata (1.5 kB)
  Using cached numba-0.61.2-cp312-cp312-manylinux2014_x86_64.manylinux_2_17_x86_64.whl.metadata (2.8 kB)
  Using cached numpy-2.2.4-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (62 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 562.6/562.6 kB 40.4 MB/s eta 0:00:00
  Using cached pan

In [1]:
# Restart the kernel to apply the changes
%reset -f

In [1]:
# import packages
import pandas as pd
import numpy as np
import cv2 as cv
import os
import time

from cuml.metrics import accuracy_score
from cuml.model_selection import train_test_split
from cuml.preprocessing import StandardScaler
from cuml.pipeline import Pipeline
from cuml.svm import SVC
from skimage.feature import hog
from sklearn.base import BaseEstimator, TransformerMixin

In [2]:
# Fetch dataset
# Suggested dir tree:
# .
# ├── submission.ipynb
# ├── train_ims
# ├── test_ims
# ├── train.csv
# ├── test.csv
dir = os.getcwd()
df = pd.read_csv('train.csv')

images = []
labels = []

for index, row in df.iterrows():
    img_path = os.path.join(dir, 'train_ims', row['im_name'])
    try:
        image = cv.imread(img_path, cv.IMREAD_ANYCOLOR)
    except:
        print(f"image {img_path} not found")
    # Add the flipeed images to augment the dataset
    images.append(cv.flip(image, 1))
    images.append(cv.flip(image, 0))
    images.append(cv.flip(image, -1))
    images.append(image)
    for i in range(4): labels.append(row['label'])

images = np.array(images)
labels = np.array(labels)
# X_train, X_test, y_train, y_test = train_test_split(images, labels, test_size=0.2, random_state=42)

In [5]:
class Preprocessor(BaseEstimator, TransformerMixin):
    """
    This preprocessor aims to strengthen the edge details of the image 
    by combining HOG information from RGB channels respectively and 
    the original image.
    """
    def __init__(self):
        super().__init__()
    def fit(self, X, y=None):
        return self
    def transform(self, X):
        features = []
        for image in X:
            r, g, b = cv.split(image)
            hog_r = hog(r, orientations=9, pixels_per_cell=(8, 8), cells_per_block=(2, 2), visualize=False)
            hog_g = hog(g, orientations=9, pixels_per_cell=(8, 8), cells_per_block=(2, 2), visualize=False)
            hog_b = hog(b, orientations=9, pixels_per_cell=(8, 8), cells_per_block=(2, 2), visualize=False)
            hog_features = np.concatenate((hog_r, hog_g, hog_b))

            image = cv.resize(image, (16,16))
            feature = np.hstack((image.flatten(), hog_features))
            features.append(feature)
        features = np.array(features)
        print(features.shape)
        return features

In [6]:
# Intialize the pipeline for training
pipeline = Pipeline([
    ('preprocessor', Preprocessor()),
    ('scaler', StandardScaler()),
    ('classifier', SVC(C=30, kernel='rbf', gamma='scale', random_state=42)),
])

In [7]:
# Training
start_time = time.time()
# pipeline.fit(X_train, y_train)
pipeline.fit(images, labels)
end_time = time.time()

print("Full training completed.")
print(f"Training time: {end_time - start_time} seconds")
print()

(200000, 1740)


: 

In [ ]:
# Predicting
test_df = pd.read_csv('test.csv')
test_images = []
test_names = []

for index, row in test_df.iterrows():
    img_path = os.path.join(dir, 'test_ims', row['im_name'])
    try:
        image = cv.imread(img_path, cv.IMREAD_ANYCOLOR)
    except:
        print(f"image {img_path} not found")
    test_images.append(image)
    test_names.append(row['im_name'])

X_test = np.array(test_images)

start_time = time.time()
y_test_pred = pipeline.predict(X_test)
end_time = time.time()

In [ ]:
# Save the result to submission.csv
output_df = pd.DataFrame({
    'im_name': test_names,
    'label': y_test_pred
})
output_df.to_csv('submission.csv', index=False)
print("submission.csv saved.")
print()